In [1]:
import win32com.client
import os
from datetime import datetime, timedelta
from dataclasses import dataclass
import pandas as pd

outlook = win32com.client.Dispatch('outlook.application')
mapi = outlook.GetNamespace('MAPI')

inbox = mapi.GetDefaultFolder(6)
archive = mapi.GetDefaultFolder(39)
inbox_messages = inbox.Items
archive_messages = archive.Items

In [2]:
messages = [inbox_messages, archive_messages]

In [3]:
'''
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")
for i in range(50):
    try:print(i,namespace.GetDefaultFolder(i).Name)
    except:pass
'''

'\noutlook = win32com.client.Dispatch("Outlook.Application")\nnamespace = outlook.GetNamespace("MAPI")\nfor i in range(50):\n    try:print(i,namespace.GetDefaultFolder(i).Name)\n    except:pass\n'

In [4]:
result_list = []
error_list = []
limit_date = '31-05-22' #doesnt include this date

for j, msg_category in enumerate(messages):

#Email address
    for index, message in enumerate(msg_category):
        try:
            if message.Class == 43:
                sender = message.Sender
                if message.SenderEmailType == 'EX':
                    if message.Sender.GetExchangeUser() != None:
                        sender_address = message.Sender.GetExchangeUser().PrimarySmtpAddress
                    else:
                        pass
                else:
                    sender_address = message.SenderEmailAddress
            cc = message.CC
            date =  message.SentOn.strftime("%d-%m-%y")
            if date == limit_date:
                break

            result_list.append([sender, sender_address, cc, date])
            print(index, date)

        except Exception as e:
            print(e)
            error_list.append(message)
            continue

result_df = pd.DataFrame(result_list, columns = ['Sender', 'Sender Address', 'CC', 'date'])

0 19-09-22
1 19-09-22
2 19-09-22
3 19-09-22
4 19-09-22
5 19-09-22
6 19-09-22
7 19-09-22
8 19-09-22
9 19-09-22
10 19-09-22
11 19-09-22
12 19-09-22
13 19-09-22
14 19-09-22
15 19-09-22
16 18-09-22
17 18-09-22
18 18-09-22
19 17-09-22
20 17-09-22
21 17-09-22
22 17-09-22
23 16-09-22
24 16-09-22
25 16-09-22
26 16-09-22
27 16-09-22
28 15-09-22
29 15-09-22
30 15-09-22
31 15-09-22
32 15-09-22
33 15-09-22
34 15-09-22
35 15-09-22
36 15-09-22
37 15-09-22
38 15-09-22
39 15-09-22
40 15-09-22
41 15-09-22
42 15-09-22
43 15-09-22
44 14-09-22
45 14-09-22
46 14-09-22
47 14-09-22
48 14-09-22
49 14-09-22
50 14-09-22
51 14-09-22
52 14-09-22
53 14-09-22
54 14-09-22
55 14-09-22
56 14-09-22
57 14-09-22
58 14-09-22
59 14-09-22
60 14-09-22
61 14-09-22
62 13-09-22
63 13-09-22
64 13-09-22
65 13-09-22
66 13-09-22
67 13-09-22
68 13-09-22
69 13-09-22
70 13-09-22
71 13-09-22
72 13-09-22
73 13-09-22
74 13-09-22
75 13-09-22
76 13-09-22
77 13-09-22
78 13-09-22
79 12-09-22
80 12-09-22
81 11-09-22
82 10-09-22
83 10-09-22
84

In [5]:
#Group
result_df['Group'] = result_df['Sender Address'].apply(lambda x: x[x.find('@')+1:])
count_group_df = result_df.copy()
count_group_df.drop_duplicates(subset=['Sender Address'], keep='first', inplace=True)

count_group_output = count_group_df.groupby('Group').count()['Sender'].reset_index() #required

In [6]:
import re

result_df['Sender_modified'] = result_df['Sender Address'].apply(lambda x: str(x)[:str(x).find('@')])
result_df['Est. Human Sender'] = result_df['Sender_modified'].apply(lambda x: x if re.match('^([a-z]+)$', x) != None else "")
result_df.drop(columns='Sender_modified', inplace=True)

In [7]:
result_df.to_excel('Information List.xlsx',index=False)
count_group_output.to_excel('Emails per group.xlsx',index=False)